<a href="https://colab.research.google.com/github/rytzi/Durian-Ripeness-Detection-Model/blob/main/Gas_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow

In [5]:
import pandas as pd

sensor_readings_df = pd.read_excel('/content/Aroma Training Data.xlsx', sheet_name='Sensor Readings', header=None).T
ripeness_labels_df = pd.read_excel('/content/Aroma Training Data.xlsx', sheet_name='Ripeness Labels', header=None)

sensor_readings_ripe = sensor_readings_df.values.tolist()

print(sensor_readings_df.shape)

(60, 20)


In [8]:
import pandas as pd

ripeness_labels_dict = dict(ripeness_labels_df.values)

ripe_readings = []
unripe_readings = []

for index, label in ripeness_labels_dict.items():
    if label == 'Ripe':
        ripe_readings.append(sensor_readings_df[index].tolist())
    elif label == 'Unripe':
        unripe_readings.append(sensor_readings_df[index].tolist())

print("Ripe Readings:")
print(ripe_readings)

print("\nUnripe Readings:")
print(unripe_readings)

Ripe Readings:
[[1133, 1031, 939, 1451, 944, 983, 1245, 1217, 1134, 1150, 954, 816, 813, 839, 929, 768, 813, 720, 975, 833], [1163, 1207, 1098, 1478, 1045, 1069, 1247, 1213, 1133, 1153, 1023, 894, 868, 930, 1023, 834, 800, 732, 1001, 859], [1190, 1311, 1175, 1441, 995, 1104, 1250, 1209, 1132, 1156, 1061, 938, 901, 959, 1060, 870, 848, 784, 1021, 874], [1222, 1363, 1236, 1429, 1072, 1129, 1255, 1205, 1131, 1159, 1079, 960, 915, 973, 1005, 837, 818, 752, 1035, 885], [1231, 1392, 1263, 1457, 1019, 1140, 1258, 1201, 1130, 1162, 1085, 975, 929, 978, 1079, 895, 878, 797, 1042, 892], [1232, 1222, 1276, 1447, 1084, 1149, 1260, 1197, 1129, 1165, 1086, 979, 935, 981, 1063, 866, 840, 801, 1048, 897], [1247, 1403, 1279, 1402, 1089, 1151, 1265, 1193, 1128, 1168, 1087, 983, 938, 976, 1068, 889, 881, 767, 1052, 900], [1239, 1405, 1280, 1446, 1091, 1150, 1268, 1189, 1127, 1171, 1083, 987, 937, 987, 1083, 900, 847, 796, 1056, 902], [1241, 1402, 1279, 1449, 1091, 1150, 1272, 1185, 1126, 1174, 1079, 987,

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import GridSearchCV

X_ripe = np.array(ripe_readings)
X_unripe = np.array(unripe_readings)

y_ripe = np.ones(len(ripe_readings))
y_unripe = np.zeros(len(unripe_readings))

X = np.concatenate((X_ripe, X_unripe), axis=0)
y = np.concatenate((y_ripe, y_unripe))

indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

model = Sequential([
    Dense(128, activation='relu', input_shape=(len(X_train[0]),)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/100
1/1 [==============================] - 1s 763ms/step - loss: 476.0482 - accuracy: 0.6250 - val_loss: 350.0267 - val_accuracy: 0.2500
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 410.3404 - accuracy: 0.5625 - val_loss: 281.8433 - val_accuracy: 0.2500
Epoch 3/100
1/1 [==============================] - 0s 25ms/step - loss: 296.6416 - accuracy: 0.5000 - val_loss: 223.5950 - val_accuracy: 0.2500
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 349.7037 - accuracy: 0.6875 - val_loss: 173.2024 - val_accuracy: 0.2500
Epoch 5/100
1/1 [==============================] - 0s 24ms/step - loss: 313.5188 - accuracy: 0.5000 - val_loss: 127.6691 - val_accuracy: 0.2500
Epoch 6/100
1/1 [==============================] - 0s 26ms/step - loss: 53.5302 - accuracy: 0.6875 - val_loss: 88.5173 - val_accuracy: 0.2500
Epoch 7/100
1/1 [==============================] - 0s 26ms/step - loss: 103.4373 - accuracy: 0.6250 - val_loss: 58.7578 - val_accuracy: 0

In [12]:
import tensorflow as tf

# Save the model
model.save("ann_durio.h5")

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('ann_model.tflite', 'wb') as f:
    f.write(tflite_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


.
.
.
.
.
.
.

.
.
.
.
.
.
.

.
.
.
.
.
.
.
.

